#MARKET BASKET ANAYSIS


#DATA LOADING

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=929426aaaac74bfdd6b86a2bbffb4213e58c82f92690e706ebbac3ded8b660db
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MarketBasketAnalysis").getOrCreate()

In [ ]:
df_1 = spark.read.csv("/content/2010-2011-1.csv",header=True,inferSchema=True)
df_2 = spark.read.csv("/content/2009-2010-1.csv",header=True,inferSchema=True)

In [ ]:
df_1.show()

+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
| 536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26| 2.55|      17850|United Kingdom|
| 536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26| 2.75|      17850|United Kingdom|
| 536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 08:26| 7.65|      17850|United Kingdom|
| 536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 08:26| 4.25|      17850|United Kingdom|


In [ ]:
df_2.show()

+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|01-12-2009 07:45| 6.95|      13085|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|01-12-2009 07:45| 6.75|      13085|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|01-12-2009 07:45| 6.75|      13085|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|01-12-2009 07:45|  2.1|      13085|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|01-12-2009 07:45| 1.25|      13085|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|01-12-2009 07:45| 1.65|      13085|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|01-12-2009 07:45| 1.25|      13085|United Kingdom|


#PREPROCESSING  


DATA EXPLORATION

In [ ]:
df_1.describe().show()

+-------+------------------+------------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|summary|           Invoice|         StockCode|         Description|          Quantity|     InvoiceDate|            Price|       Customer ID|    Country|
+-------+------------------+------------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|  count|            541910|            541910|              540456|            541910|          541910|           541910|            406830|     541910|
|   mean| 559965.7926209917|27623.240210938104|             20713.0| 9.552233765754462|            NULL|4.611138332927965|15287.684160460143|       NULL|
| stddev|13428.437355340984| 16799.73762842774|                NULL|218.08095694392543|            NULL|96.75976549366548|1713.6030743514768|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|        

In [ ]:
df_2.describe().show()

+-------+------------------+-----------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|summary|           Invoice|        StockCode|         Description|          Quantity|     InvoiceDate|            Price|       Customer ID|    Country|
+-------+------------------+-----------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|  count|            525461|           525461|              522533|            525461|          525461|           525461|            417534|     525461|
|   mean| 514496.9241788484|30518.37992495103|  22226.666666666668|10.337667305470815|            NULL| 4.68883447867188|15360.645477973052|       NULL|
| stddev|14439.209494472732|20693.16815285475|   646.8974674037095|107.42411026879805|            NULL|146.1269139529275|1680.8113157457294|       NULL|
|    min|            489434|            10002|  DOORMAT UNION J...|             -9

In [ ]:
df_1.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('Price', 'double'),
 ('Customer ID', 'int'),
 ('Country', 'string')]

In [ ]:
df_2.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('Price', 'double'),
 ('Customer ID', 'int'),
 ('Country', 'string')]

Attribute Information:

InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.


StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.


Description: Product (item) name. Nominal.
Quantity: The quantities of each product (item) per transaction. Numeric.
InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.


UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).


CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.


Country: Country name. Nominal. The name of the country where a customer resides.

COLUMN RENAME

In [ ]:
df_1.columns

['Invoice',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'Price',
 'Customer ID',
 'Country']

In [ ]:
df_2.columns

['Invoice',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'Price',
 'Customer ID',
 'Country']

In [ ]:
df_1 = df_1.withColumnRenamed("Invoice","transaction_id")
df_1 = df_1.withColumnRenamed("StockCode","product_id")
df_1 = df_1.withColumnRenamed("Description","product_name")
df_1 = df_1.withColumnRenamed("Quantity","quantity")
df_1 = df_1.withColumnRenamed("InvoiceDate","transaction_date")
df_1 = df_1.withColumnRenamed("Price","price")
df_1 = df_1.withColumnRenamed("Customer ID","customer_id")
df_1 = df_1.withColumnRenamed("Country","country")

In [ ]:
df_1.columns

['transaction_id',
 'product_id',
 'product_name',
 'quantity',
 'transaction_date',
 'price',
 'customer_id',
 'country']

In [ ]:
df_2 = df_2.withColumnRenamed("Invoice","transaction_id")
df_2 = df_2.withColumnRenamed("StockCode","product_id")
df_2 = df_2.withColumnRenamed("Description","product_name")
df_2 = df_2.withColumnRenamed("Quantity","quantity")
df_2 = df_2.withColumnRenamed("InvoiceDate","transaction_date")
df_2 = df_2.withColumnRenamed("Price","price")
df_2 = df_2.withColumnRenamed("Customer ID","customer_id")
df_2 = df_2.withColumnRenamed("Country","country")

In [ ]:
df_2.columns

['transaction_id',
 'product_id',
 'product_name',
 'quantity',
 'transaction_date',
 'price',
 'customer_id',
 'country']

DEALING WITH NULL VALUES

In [ ]:
print((df_1.count(), len(df_1.columns)))

(541910, 8)


In [ ]:
df_1 = df_1.dropna()

In [ ]:
print((df_1.count(), len(df_1.columns)))

(406830, 8)


In [ ]:
print((df_2.count(), len(df_2.columns)))

(525461, 8)


In [ ]:
df_2 = df_2.dropna()

In [ ]:
print((df_2.count(), len(df_2.columns)))

(417534, 8)


REMOVING DUPLICATES


In [ ]:
print((df_1.count(), len(df_1.columns)))

(406830, 8)


In [ ]:
df_1 = df_1.dropDuplicates()

In [ ]:
print((df_1.count(), len(df_1.columns)))

(401605, 8)


In [ ]:
print((df_2.count(), len(df_2.columns)))

(417534, 8)


In [ ]:
df_2 = df_2.dropDuplicates()

In [ ]:
print((df_2.count(), len(df_2.columns)))

(410763, 8)


IN Column 'Transation id' removing rows staring with 'C' SINCE they are cancelled order

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df_1 = df_1.filter(~col("transaction_id").startswith("C"))
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
print((df_1.count(), len(df_1.columns)))

(392733, 8)


In [ ]:
df_2 = df_2.filter(~col("transaction_id").startswith("C"))
df_2.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        489436|     21756|BATH BUILDING BLO...|       3|01-12-2009 09:06| 5.95|      13078|United Kingdom|
|        489438|    84519A|TOMATO CHARLIE+LO...|      56|01-12-2009 09:24| 2.15|      18102|United Kingdom|
|        489446|     22199|FRYING PAN RED PO...|       4|01-12-2009 10:06| 4.25|      13758|United Kingdom|
|        489520|     22083|PAPER CHAIN KIT R...|       6|01-12-2009 11:41| 2.95|      14911|          EIRE|
|        489522|     21133|PAINTED BIRD ASSO...|       6|01-12-2009 11:45| 1.69|      15998|United Kingdom|
|        489529|     21500|   PINK SPOTS  WRAP |      25|01-12-2009 11:51| 0.42|      17984|United Kingdom|
|        489545|     22113|G

In [ ]:
print((df_2.count(), len(df_2.columns)))

(400947, 8)


converting column 'TRANSACTION_ID' to INT

In [ ]:
df_1 = df_1.withColumn("transaction_id", df_1["transaction_id"].cast("int"))
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
df_2 = df_2.withColumn("transaction_id", df_2["transaction_id"].cast("int"))
df_2.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        489436|     21756|BATH BUILDING BLO...|       3|01-12-2009 09:06| 5.95|      13078|United Kingdom|
|        489438|    84519A|TOMATO CHARLIE+LO...|      56|01-12-2009 09:24| 2.15|      18102|United Kingdom|
|        489446|     22199|FRYING PAN RED PO...|       4|01-12-2009 10:06| 4.25|      13758|United Kingdom|
|        489520|     22083|PAPER CHAIN KIT R...|       6|01-12-2009 11:41| 2.95|      14911|          EIRE|
|        489522|     21133|PAINTED BIRD ASSO...|       6|01-12-2009 11:45| 1.69|      15998|United Kingdom|
|        489529|     21500|   PINK SPOTS  WRAP |      25|01-12-2009 11:51| 0.42|      17984|United Kingdom|
|        489545|     22113|G

In [ ]:
df_2.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

Spliting 'transaction_date' column to 'transaction_date' and 'transaction_time'

In [ ]:
df_1 = df_1
df_2 = df_2

In [ ]:
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
from pyspark.sql.functions import substring,to_date

df_1 = df_1.withColumn('transaction_date', to_date(substring(df_1['transaction_date'], 1, 10), 'dd-MM-yyyy'))

df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|      2010-12-01| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|      2010-12-01|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|      2010-12-01| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|      2010-12-01| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|      2010-12-01| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|      2010-12-01| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'date'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
df_2.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        489436|     21756|BATH BUILDING BLO...|       3|01-12-2009 09:06| 5.95|      13078|United Kingdom|
|        489438|    84519A|TOMATO CHARLIE+LO...|      56|01-12-2009 09:24| 2.15|      18102|United Kingdom|
|        489446|     22199|FRYING PAN RED PO...|       4|01-12-2009 10:06| 4.25|      13758|United Kingdom|
|        489520|     22083|PAPER CHAIN KIT R...|       6|01-12-2009 11:41| 2.95|      14911|          EIRE|
|        489522|     21133|PAINTED BIRD ASSO...|       6|01-12-2009 11:45| 1.69|      15998|United Kingdom|
|        489529|     21500|   PINK SPOTS  WRAP |      25|01-12-2009 11:51| 0.42|      17984|United Kingdom|
|        489545|     22113|G

In [ ]:
df_2.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
from pyspark.sql.functions import substring

df_2 = df_2.withColumn('transaction_date', to_date(substring(df_2['transaction_date'], 1, 10), 'dd-MM-yyyy'))

df_2.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        489436|     21756|BATH BUILDING BLO...|       3|      2009-12-01| 5.95|      13078|United Kingdom|
|        489438|    84519A|TOMATO CHARLIE+LO...|      56|      2009-12-01| 2.15|      18102|United Kingdom|
|        489446|     22199|FRYING PAN RED PO...|       4|      2009-12-01| 4.25|      13758|United Kingdom|
|        489520|     22083|PAPER CHAIN KIT R...|       6|      2009-12-01| 2.95|      14911|          EIRE|
|        489522|     21133|PAINTED BIRD ASSO...|       6|      2009-12-01| 1.69|      15998|United Kingdom|
|        489529|     21500|   PINK SPOTS  WRAP |      25|      2009-12-01| 0.42|      17984|United Kingdom|
|        489545|     22113|G

In [ ]:
df_2.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'date'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

#MARKET BASKET ANALYSIS ALGORITHMS

FP-GROWTH ALGORITHM

Apriori algorithm generates all itemsets by scanning the full transactional database. Whereas the FP growth algorithm only generates the frequent itemsets according to the minimum support defined by the user.
FP(Frequent Pattern) Tree is better than Apriori Algorithm.

In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F

In [ ]:
basketdata = df_1.dropDuplicates(['transaction_id','product_name']).sort('transaction_id')
original_data=basketdata
basketdata.show()
print((basketdata.count(), len(basketdata.columns)))

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536365|    84406B|CREAM CUPID HEART...|       8|      2010-12-01| 2.75|      17850|United Kingdom|
|        536365|     21730|GLASS STAR FROSTE...|       6|      2010-12-01| 4.25|      17850|United Kingdom|
|        536365|    84029G|KNITTED UNION FLA...|       6|      2010-12-01| 3.39|      17850|United Kingdom|
|        536365|    85123A|WHITE HANGING HEA...|       6|      2010-12-01| 2.55|      17850|United Kingdom|
|        536365|    84029E|RED WOOLLY HOTTIE...|       6|      2010-12-01| 3.39|      17850|United Kingdom|
|        536365|     22752|SET 7 BABUSHKA NE...|       2|      2010-12-01| 7.65|      17850|United Kingdom|
|        536365|     71053| 

In [ ]:
basketdata = basketdata.groupBy("transaction_id").agg(F.collect_list("product_name")).sort('transaction_id')
basketdata.show()

+--------------+--------------------------+
|transaction_id|collect_list(product_name)|
+--------------+--------------------------+
|        536365|      [SET 7 BABUSHKA N...|
|        536366|      [HAND WARMER UNIO...|
|        536367|      [ASSORTED COLOUR ...|
|        536368|      [BLUE COAT RACK P...|
|        536369|      [BATH BUILDING BL...|
|        536370|      [ROUND SNACK BOXE...|
|        536371|      [PAPER CHAIN KIT ...|
|        536372|      [HAND WARMER UNIO...|
|        536373|      [WOODEN FRAME ANT...|
|        536374|      [VICTORIAN SEWING...|
|        536375|      [WOODEN PICTURE F...|
|        536376|      [HOT WATER BOTTLE...|
|        536377|      [HAND WARMER UNIO...|
|        536378|      [PACK OF 60 DINOS...|
|        536380|      [JAM MAKING SET P...|
|        536381|      [GUMBALL MONOCHRO...|
|        536382|      [ANTIQUE GLASS DR...|
|        536384|      [ENAMEL FLOWER JU...|
|        536385|      [TRADITIONAL CHRI...|
|        536386|      [JUMBO BAG

In [ ]:
basketdata

DataFrame[transaction_id: int, collect_list(product_name): array<string>]

In [ ]:
basketdata = basketdata.withColumnRenamed("collect_list(product_name)","Items")
basketdata.show()

+--------------+--------------------+
|transaction_id|               Items|
+--------------+--------------------+
|        536365|[SET 7 BABUSHKA N...|
|        536366|[HAND WARMER UNIO...|
|        536367|[ASSORTED COLOUR ...|
|        536368|[BLUE COAT RACK P...|
|        536369|[BATH BUILDING BL...|
|        536370|[ROUND SNACK BOXE...|
|        536371|[PAPER CHAIN KIT ...|
|        536372|[HAND WARMER UNIO...|
|        536373|[WOODEN FRAME ANT...|
|        536374|[VICTORIAN SEWING...|
|        536375|[WOODEN PICTURE F...|
|        536376|[HOT WATER BOTTLE...|
|        536377|[HAND WARMER UNIO...|
|        536378|[PACK OF 60 DINOS...|
|        536380|[JAM MAKING SET P...|
|        536381|[GUMBALL MONOCHRO...|
|        536382|[ANTIQUE GLASS DR...|
|        536384|[ENAMEL FLOWER JU...|
|        536385|[TRADITIONAL CHRI...|
|        536386|[JUMBO BAG RED RE...|
+--------------+--------------------+
only showing top 20 rows



In [ ]:
fpGrowth = FPGrowth(itemsCol="Items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(basketdata)

minSupport: It sets the minimum support threshold. The minSupport parameter is a fraction between 0 and 1, and it represents the minimum proportion of transactions that must contain a particular pattern (a set of items) for it to be considered frequent. In this case, the threshold is set to 0.006, which means that a pattern must appear in at least 0.6% of the transactions to be considered frequent.


minConfidence: It sets the minimum confidence threshold for generating association rules. The minConfidence parameter is also a fraction between 0 and 1. It represents the minimum level of confidence required for an association rule to be generated. In this case, it's set to 0.006, which corresponds to 0.6% confidence.

In [ ]:
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[12 COLOURED PART...| 139|
|[TRAVEL CARD WALL...| 205|
|[TRAVEL CARD WALL...| 119|
|[JUMBO BAG SCANDI...| 451|
|[JUMBO BAG SCANDI...| 112|
|[JUMBO BAG SCANDI...| 124|
|[JUMBO BAG SCANDI...| 128|
|[JUMBO BAG SCANDI...| 134|
|[JUMBO BAG SCANDI...| 122|
|[JUMBO BAG SCANDI...| 268|
|[JUMBO BAG SCANDI...| 134|
|[JUMBO BAG SCANDI...| 241|
|[JUMBO BAG SCANDI...| 159|
|[JUMBO BAG SCANDI...| 125|
|[JUMBO BAG SCANDI...| 126|
|[JUMBO BAG SCANDI...| 198|
|[JUMBO BAG SCANDI...| 143|
|[JUMBO BAG SCANDI...| 151|
|[JUMBO BAG SCANDI...| 113|
|[JUMBO BAG SCANDI...| 155|
+--------------------+----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import desc

sorted_data = model.freqItemsets.sort(desc("freq"))
sorted_data.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[WHITE HANGING HE...|1971|
|[REGENCY CAKESTAN...|1704|
|[JUMBO BAG RED RE...|1600|
|     [PARTY BUNTING]|1380|
|[ASSORTED COLOUR ...|1375|
|[LUNCH BAG RED RE...|1288|
|[SET OF 3 CAKE TI...|1146|
|           [POSTAGE]|1100|
|[LUNCH BAG  BLACK...|1052|
|[PACK OF 72 RETRO...|1029|
|    [SPOTTY BUNTING]|1003|
|[LUNCH BAG SPACEB...| 988|
|[PAPER CHAIN KIT ...| 980|
|[NATURAL SLATE HE...| 967|
|[LUNCH BAG CARS B...| 966|
|[HEART OF WICKER ...| 961|
|[LUNCH BAG PINK P...| 931|
|[LUNCH BAG SUKI D...| 899|
|[JAM MAKING SET W...| 879|
|[ALARM CLOCK BAKE...| 877|
+--------------------+----+
only showing top 20 rows



In [ ]:
model.associationRules.show()

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[FELTCRAFT GIRL N...|[FELTCRAFT GIRL A...| 0.7529411764705882| 54.73144175317186|0.006905481225722...|
|[JUMBO BAG ALPHAB...|[JUMBO BAG VINTAG...|0.47297297297297297|11.356252625682677|0.007552870090634...|
|[JUMBO BAG ALPHAB...|[JUMBO STORAGE BA...| 0.4155405405405405| 9.951498009637543|0.006635735865343116|
|[JUMBO BAG ALPHAB...|[JUMBO BAG PINK P...|0.46621621621621623| 9.921680578396996|0.007444971946482521|
|[JUMBO BAG ALPHAB...|  [JUMBO BAG APPLES]| 0.4864864864864865|12.559211021606565|0.007768666378938282|
|[SET OF 6 SNACK L...|[SET OF 6 TEA TIM...| 0.6683168316831684|38.833607498680905| 0.00728312473025464|
|[SET OF 6 SNACK L...|[SET OF 12 MINI L...| 0.7425742574257426|3

Association rules describe relationships between items and can be used to understand how frequently items are purchased together and the strength of those associations.

1. antecedent: which are the items typically found together before a purchase.

2. consequent: which are the items that are likely to be purchased if the antecedent items are present.

3. confidence: This is the confidence value, which indicates the probability that the consequent will be purchased when the antecedent is present.

4. lift: The lift value quantifies how much more likely the consequent is to be purchased when the antecedent is present, compared to when the consequent is purchased independently of the antecedent. A lift value greater than 1 indicates a positive association.

5. support: The support count, which represents how frequently the association rule occurs in the dataset.

In [ ]:
rules = model.associationRules
model.transform(basketdata).show()

+--------------+--------------------+--------------------+
|transaction_id|               Items|          prediction|
+--------------+--------------------+--------------------+
|        536365|[SET 7 BABUSHKA N...|[LARGE WHITE HEAR...|
|        536366|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536367|[ASSORTED COLOUR ...|[DOORMAT ENGLISH ...|
|        536368|[BLUE COAT RACK P...|[HOMEMADE JAM SCE...|
|        536369|[BATH BUILDING BL...|[HOME BUILDING BL...|
|        536370|[ROUND SNACK BOXE...|[ALARM CLOCK BAKE...|
|        536371|[PAPER CHAIN KIT ...|[HAND WARMER OWL ...|
|        536372|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536373|[WOODEN FRAME ANT...|[WOOD BLACK BOARD...|
|        536374|[VICTORIAN SEWING...|                  []|
|        536375|[WOODEN PICTURE F...|[WOOD BLACK BOARD...|
|        536376|[HOT WATER BOTTLE...|[CANDLEHOLDER PIN...|
|        536377|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536378|[PACK OF 60 DINOS...|[GINGERBREAD MAN ..

In [ ]:
transformed = model.transform(basketdata)
transformed.show()

+--------------+--------------------+--------------------+
|transaction_id|               Items|          prediction|
+--------------+--------------------+--------------------+
|        536365|[SET 7 BABUSHKA N...|[LARGE WHITE HEAR...|
|        536366|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536367|[ASSORTED COLOUR ...|[DOORMAT ENGLISH ...|
|        536368|[BLUE COAT RACK P...|[HOMEMADE JAM SCE...|
|        536369|[BATH BUILDING BL...|[HOME BUILDING BL...|
|        536370|[ROUND SNACK BOXE...|[ALARM CLOCK BAKE...|
|        536371|[PAPER CHAIN KIT ...|[HAND WARMER OWL ...|
|        536372|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536373|[WOODEN FRAME ANT...|[WOOD BLACK BOARD...|
|        536374|[VICTORIAN SEWING...|                  []|
|        536375|[WOODEN PICTURE F...|[WOOD BLACK BOARD...|
|        536376|[HOT WATER BOTTLE...|[CANDLEHOLDER PIN...|
|        536377|[HAND WARMER UNIO...|[HAND WARMER OWL ...|
|        536378|[PACK OF 60 DINOS...|[GINGERBREAD MAN ..